_Authors: Andreia Dourado, Bruno Moraes_

_Adapted from Melissa Graham example notebook \<details later\> and discussion in Rubin Community Forum._

__Description: This is a first notebook to get a random galaxy sample from the Rubin Science Platform DP0.2 data to be used as a photo-z training set.__


### 1. Importing packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from lsst.rsp import get_tap_service, retrieve_query

### 2. Query 

In [ ]:
pd.set_option('display.max_rows', 200)
%matplotlib inline
service = get_tap_service("tap")

##### 2.1 Selecting a random objectIds fraction

In [ ]:
filename = '/scratch/andreiadourado/dp02_dc2_catalogs_Object_all_objectids_sorted.csv'
random.seed(4321)
random_ids = pd.read_csv(filename, 
           skiprows=lambda x: x > 0 and random.random() >=0.006) 

In [ ]:
len(random_ids)

In [ ]:
random_ids.to_csv('random_ids1.csv')

Due to a limit of data that can be obtained on RSP, it was possible to obtain 50000 objects in each query. The following steps were repeated until to obtain all data. 

#### 2.2 Writting the objectIds in the query format

In [ ]:
ids = np.squeeze((random_ids.values)[50000:100000])
len(ids)

In [ ]:
obj_ids = str(tuple(ids))

#### 2.3 JOIN Object table and TruthSummary table

In [ ]:
%%time
query = '\n'.join(['SELECT mt.id_truth_type, mt.match_objectId, obj.objectId, ts.ra, ts.dec, ts.redshift',
        'FROM dp02_dc2_catalogs.MatchesTruth AS mt ',
        'JOIN dp02_dc2_catalogs.TruthSummary AS ts ',
        'ON mt.id_truth_type=ts.id_truth_type ',
        'JOIN dp02_dc2_catalogs.Object AS obj ',
        'ON mt.match_objectId=obj.objectId ', 
        'WHERE obj.objectId IN'+obj_ids,
        'AND ts.truth_type=1 ',
        'AND obj.detect_isPrimary=1'])

#print(query)
#print(' ')
#results = service.search(query).to_table().to_pandas()#converte para tabela astropy e depois pra pandas dataframe
#print('len(results) = ', len(results))

Due to a time limit imposed on synchronous query, it was suggested the following asynchronous query:

In [ ]:
job = service.submit_job(query)
print('Job URL is', job.url)
print('Job phase is', job.phase)

This run take about 20 minutes.

In [ ]:
%%time
job.run()

In [ ]:
print('Job phase is now ', job.phase) 

In [ ]:
async_results = job.fetch_result().to_table().to_pandas()

In [ ]:
len(async_results)

In [ ]:
async_results.to_csv("random_data_1.csv")

On-sky spatial distribuition

In [ ]:
plt.scatter(async_results['ra'],async_results['dec'], c=async_results['redshift'],cmap='turbo',s=1)
plt.colorbar()

#### 4. Final sample

Join all query results in only data frame:

In [ ]:
num = #quantidade de queries rodadas + 1 
filenames = ["DadosRandom" + str(i) + ".csv" for i in range(1, num)] 

In [ ]:
data = pd.DataFrame()
for filename in filenames:
    df = pd.read_csv(filename)
    pd.concat([data, df])

In [ ]:
data_specz=data[['objectId','ra','dec','redshift']]

On-sky spatial distribuition

In [ ]:
plt.scatter(data_specz['ra'],data_specz['dec'], c=data_specz['redshift'],cmap='turbo',s=1)
plt.colorbar()

In [ ]:
data_specz.to_csv("specz_data.csv")

In [ ]:
data.to_csv('random_data.csv') 